In [1]:
import glob
import os
import pandas as pd
import yaml
import re

In [2]:
filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))

/tmp/ipykernel_211175/4175686493.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))


In [3]:
dataset = "twitter"

filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(filepaths[f"04_{dataset}_scored"], "*"))]

In [4]:
df_swearings = pd.read_csv(filepaths["00_bad_words_scored"])
swearing_set = set(df_swearings.swearing.values)

for filename in filenames:
    df = pd.read_csv(os.path.join(filepaths[f"04_{dataset}_scored"], filename))
    
    df["text"] = df.text.str.lower()
    
    df["has_swearing"] = False
    for i, row in df.iterrows():
        splitted_text = re.sub("[.,;:?!]+", '', row.text).split()
        intersection = swearing_set.intersection(splitted_text)
        if len(intersection):
            df.loc[i, "has_swearing"] = True
    
    df = df[~df.text.str.contains("__fuckin__")]
    df.to_csv(os.path.join(filepaths[f"04_{dataset}_scored"], filename), index=False)